# #Tek Değişkenli Outlier Baskılama

In [2]:
#data--> analiz edilecek data
#columns--> analiz edilecek numerik kolonlar
#library
import pandas as pd

#örnek girdi***********
data = {
    "highwaympg": [32, 30, 28, 35, 18],
    "citympg": [28, 25, 22, 29, 15],
    "peakrpm": [5500, 6000, 5800, 6200, 5400],
    "horsepower": [150, 170, 140, 160, 380],
    "compressionratio": [9.0, 9.5, 10.0, 8.5, 7.0]
}

df = pd.DataFrame(data)
data=df
#*******************************



def numeric_outlier_analysis(data, columns):
    for i in columns:
        df_meta = data[i]
        Q1 = df_meta.quantile(0.25)
        Q3 = df_meta.quantile(0.75)
        IQR = Q3 - Q1
        alt_sinir = abs(Q1 - 1.5 * IQR)
        ust_sinir = abs(Q3 + 1.5 * IQR)
        print("alt_sinir-->" + str(alt_sinir))
        print("ust_sinir-->" + str(ust_sinir))
        aykiri_tf_alt = (df_meta < alt_sinir)
        aykiri_tf_ust = (ust_sinir < df_meta)
        print("baskılanıyor..." + str(i))
        df_meta[aykiri_tf_alt] = alt_sinir
        df_meta[aykiri_tf_ust] = ust_sinir
        data[i]=df_meta
        
        
        
#çalıştırma bölümü 
#örnek
col = ["highwaympg", "citympg", "peakrpm", "horsepower", "compressionratio"]
        
numeric_outlier_analysis(data, col)

alt_sinir-->22.0
ust_sinir-->38.0
baskılanıyor...highwaympg
alt_sinir-->13.0
ust_sinir-->37.0
baskılanıyor...citympg
alt_sinir-->4750.0
ust_sinir-->6750.0
baskılanıyor...peakrpm
alt_sinir-->120.0
ust_sinir-->200.0
baskılanıyor...horsepower
alt_sinir-->7.0
ust_sinir-->11.0
baskılanıyor...compressionratio


C:\Users\Selman\AppData\Local\Temp\ipykernel_30668\799582577.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meta[aykiri_tf_alt] = alt_sinir
C:\Users\Selman\AppData\Local\Temp\ipykernel_30668\799582577.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meta[aykiri_tf_ust] = ust_sinir


## "object" tipini "category" ye çevirme

In [3]:
#dönüştürelecek DataFrame adı -->data

columns_l = list(data.columns)
categorical_columns = []

object_to_category = False  # Önce döngüde herhangi bir değişiklik yapılmadığını varsayalım

for i in columns_l:
    if data[i].dtype == 'object' or data[i].dtype.name == 'category':
        categorical_columns.append(i)
        if data[i].dtype == 'object':
            object_to_category = True  # En az bir "object" tipinde sütun bulundu

# "object" tipindeki sütunları "category" tipine dönüştürme ve çevrilenleri yazdırma
for column in categorical_columns:
    if data[column].dtype == 'object':
        data[column] = data[column].astype('category')

# Eğer dönüştürme yapıldıysa, çevrilenleri yazdırma
if object_to_category:
    print("Kategorik değişkene çevrilenler:")
    for column in categorical_columns:
        if data[column].dtype == 'category':
            print(column)
else:
    print("Object tipinde kolon yok.")

Object tipinde kolon yok.


## Dummy Değişken Dönüşümü

In [ ]:
import pandas as pd
#girdi--> data

df = pd.DataFrame(data)

# Kategorik sütunlara dönüşüm ve (n-1) kuralı uygulama
categorical_columns = df.select_dtypes(include=["category"]).columns

for column in categorical_columns:
    dummies = pd.get_dummies(df[column], prefix=column, drop_first=True)
    df = pd.concat([df, dummies], axis=1)
    df.drop(column, axis=1, inplace=True)
    
data=df

## oversampling under sampling

In [ ]:
df_3 = df[df.quality==3]     # MINORITY          
df_4 = df[df.quality==4]     # MINORITY          
df_5 = df[df.quality==5]     # MAJORITY
df_6 = df[df.quality==6]     # MAJORITY
df_7 = df[df.quality==7]     # MINORITY
df_8 = df[df.quality==8]     # MINORITY

#We had already established it earlier that except quality 5 and 6, all the others are minority!



# Oversample MINORITY Class to make balance data :
from sklearn.utils import resample

df_3_upsampled = resample(df_3, replace=True, n_samples=600, random_state=12) 
df_4_upsampled = resample(df_4, replace=True, n_samples=600, random_state=12) 
df_7_upsampled = resample(df_7, replace=True, n_samples=600, random_state=12) 
df_8_upsampled = resample(df_8, replace=True, n_samples=600, random_state=12) 

# Decreases the rows of Majority one's to make balance data :
df_5_downsampled = df[df.quality==5].sample(n=600).reset_index(drop=True)
df_6_downsampled = df[df.quality==6].sample(n=600).reset_index(drop=True)

## standardizasyon

In [ ]:
from sklearn.preprocessing import StandardScaler

# Veriyi alın
# x_train: eğitim verisi
# x_test: test verisi

# Seçilen dummy değişken sütunları
selected_dummy_cols = ["dummy_col_1", "dummy_col_2", "dummy_col_3"]

# Seçilen dummy değişkenleri çıkarın ve kalan sütunlar üzerinde işlem yapın
x_train_without_dummy = x_train.drop(selected_dummy_cols, axis=1)
x_test_without_dummy = x_test.drop(selected_dummy_cols, axis=1)

# StandardScaler nesnesini oluşturun
scaler = StandardScaler()

# Eğitim verisini kullanarak ölçeklendirme işlemini öğrenin ve uygulayın
x_train_scaled = scaler.fit_transform(x_train_without_dummy)

# Test verisini eğitim verisine göre aynı şekilde ölçeklendirin
x_test_scaled = scaler.transform(x_test_without_dummy)

# Dummy değişken sütunları ile ölçeklendirilmiş verileri birleştirin
x_train_scaled_df = pd.DataFrame(x_train_scaled, columns=x_train_without_dummy.columns, index=x_train_without_dummy.index)
x_test_scaled_df = pd.DataFrame(x_test_scaled, columns=x_test_without_dummy.columns, index=x_test_without_dummy.index)

# Dummy değişken sütunları ekleyerek ölçeklendirilmiş verileri tamamlanmış veriye dönüştürün
x_train_final = pd.concat([x_train_scaled_df, x_train[selected_dummy_cols]], axis=1)
x_test_final = pd.concat([x_test_scaled_df, x_test[selected_dummy_cols]], axis=1)
